# Exploring Tooling with Weights and Biases
Similar to tensorboard, weights and biases is an application that tracks all your training metrics, and performs visualizations for you. This tool allows you to cleanly sort, organize, and visualize your experiments. In this notebook, we will go through an example of how to use wandb.ai and have you practice.

1. Make an account at https://wandb.ai/site

2. pip install wandb

3. wandb login

4. After step 3, please paste your wandb API key


In [ ]:
!wget https://raw.githubusercontent.com/Berkeley-CS182/cs182fa23_public/main/q_wandbai/architectures.py
!pip install wandb

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import wandb
from architectures import BasicConvNet, ResNet18, MLP

## Organizing wandb Projects

With each run, you will want to have a set of parameters associated with it. For example, I want to be able to log different hyperparameters that I am using, so let's clearly list them below

In [ ]:
project = 'CS182 WANDB.AI Practice Notebok'
learning_rate = 0.01
epochs = 2
architecture ='CNN'
dataset = 'CIFAR-10'
batch_size = 64
momentum = 0.9
log_freq = 20
print_freq = 200
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

### Initializing the Run

In [ ]:
wandb.init(
    # set the wandb project where this run will be logged
    project=project,
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": learning_rate,
    "architecture": architecture,
    "dataset": dataset,
    "epochs": epochs,
    "batch_size": batch_size,
    "momentum": momentum
    }
)

From here on, we have some standard CIFAR training definitions.

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
net = Net()

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=momentum)

### Training with wandb

As you can see, similar to tensorboard, each gradient step we will want to log the accuracy and loss. See below for an example.

In [ ]:
for epoch in range(epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    running_acc = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        accuracy = torch.mean((torch.argmax(outputs, dim=1) == labels).float()).item() * 100

        # print statistics
        running_acc += accuracy
        running_loss += loss.item()
        if i % log_freq == log_freq - 1:
            wandb.log({'accuracy': accuracy, 'loss': loss.item()})
            
        if i % print_freq == print_freq - 1:    # print every 2000 mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / print_freq:.5f} accuracy: {running_acc/print_freq:.5f}')
            running_loss = 0.0
            running_acc = 0.0
        
            
            
        
        
            


After we are done with this run, we will want to call 
 `wandb.finish()`

In [ ]:
wandb.finish()

## Your Task
We will be once again building classifiers for the CIFAR-10. There are various architectures set up for you to use in the architectures.py file. Using wandb, please search through 10 different hyperparameter configurations. Examples of choices include: learning rate, batch size, architecture, optimization algorithm, etc. Please submit the hyperparameters that result in the highest accuracies for this classification task. Please then explore wandb for all the visualization that you may need. In addition, feel free to run as many epochs as you like.

In [ ]:
def run(params):
    raise NotImplementedError

This software/tutorial is based on PyTorch, an open-source project available at https://github.com/pytorch/tutorials/

There is a BSD 3-Clause License as seen here: https://github.com/pytorch/tutorials/blob/main/LICENSE